# **NLP Week 7 Homework**:
**Subitted by: Srashti Soni**

In [201]:
import pandas as pd
import requests
from textblob import TextBlob
import nltk
nltk.download('punkt')
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split 
from sklearn.base import TransformerMixin
from sklearn.svm import LinearSVC 
from sklearn.pipeline import Pipeline 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import LinearSVC
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [202]:
# Import Data
data = open('/content/Burbank.txt').read()
data

'\ufeffAirport task force to consider possible actions to abate noise issues Members of the Southern San Fernando Valley Airplane Noise Task Force will have a little over a month to develop recommendations about how to address aircraft noise issues affecting the south San Fernando Valley After five meetings and hearing from various neighborhood groups and experts, aviation consultant HMMH Inc., which has been the facilitator of the meetings, spent the majority of Wednesday night distilling all of the recommendations that have been brought forward so far and the feasibility of implementing them \nHMMH worked with Kevin Karpe, an aviation expert who worked as an air traffic controller at Hollywood Burbank and the Federal Aviation Administration Southern California Terminal Radar Approach Control, to analyze the departing flight procedures out of the Burbank airfield as well as Van Nuys Airport, both of which are at the center of the noise issues \nSince March 2017, residents in the south

In [203]:
# Text Cleaning
res = re.sub(r'[^\w\s]', ' ', data)
res
res=res.lower()
res

' airport task force to consider possible actions to abate noise issues members of the southern san fernando valley airplane noise task force will have a little over a month to develop recommendations about how to address aircraft noise issues affecting the south san fernando valley after five meetings and hearing from various neighborhood groups and experts  aviation consultant hmmh inc   which has been the facilitator of the meetings  spent the majority of wednesday night distilling all of the recommendations that have been brought forward so far and the feasibility of implementing them \nhmmh worked with kevin karpe  an aviation expert who worked as an air traffic controller at hollywood burbank and the federal aviation administration southern california terminal radar approach control  to analyze the departing flight procedures out of the burbank airfield as well as van nuys airport  both of which are at the center of the noise issues \nsince march 2017  residents in the south san 

**Exercise 1:**

In [204]:
blob=TextBlob(data)

In [205]:
total=0
pos=0
neg=0
neu=0
for sentence in blob.sentences:
  total +=1
  if sentence.sentiment.polarity > 0:
    pos +=1
  elif sentence.sentiment.polarity == 0:
    neu +=1
  elif sentence.sentiment.polarity < 0:
    neg +=1

print('Negative Probability:',(neg/total))

Negative Probability: 0.07142857142857142


**Exercise 2:**

In [206]:
# NaiveBayesAnalyzer
blob_object = TextBlob(data, analyzer=NaiveBayesAnalyzer())
sentiment = blob_object.sentiment
print(sentiment)

Sentiment(classification='pos', p_pos=1.0, p_neg=1.0487590501191368e-20)


**Exercise 3:**

In [207]:
tokens = nltk.word_tokenize(data)
print(tokens)

['\ufeffAirport', 'task', 'force', 'to', 'consider', 'possible', 'actions', 'to', 'abate', 'noise', 'issues', 'Members', 'of', 'the', 'Southern', 'San', 'Fernando', 'Valley', 'Airplane', 'Noise', 'Task', 'Force', 'will', 'have', 'a', 'little', 'over', 'a', 'month', 'to', 'develop', 'recommendations', 'about', 'how', 'to', 'address', 'aircraft', 'noise', 'issues', 'affecting', 'the', 'south', 'San', 'Fernando', 'Valley', 'After', 'five', 'meetings', 'and', 'hearing', 'from', 'various', 'neighborhood', 'groups', 'and', 'experts', ',', 'aviation', 'consultant', 'HMMH', 'Inc.', ',', 'which', 'has', 'been', 'the', 'facilitator', 'of', 'the', 'meetings', ',', 'spent', 'the', 'majority', 'of', 'Wednesday', 'night', 'distilling', 'all', 'of', 'the', 'recommendations', 'that', 'have', 'been', 'brought', 'forward', 'so', 'far', 'and', 'the', 'feasibility', 'of', 'implementing', 'them', 'HMMH', 'worked', 'with', 'Kevin', 'Karpe', ',', 'an', 'aviation', 'expert', 'who', 'worked', 'as', 'an', 'air'

In [208]:
sw = nltk.corpus.stopwords.words('english')
word_freq = {}
for word in nltk.word_tokenize(res):
    if word not in sw:
        if word not in word_freq.keys():
            word_freq[word] = 1
        else:
            word_freq[word] += 1

In [209]:
print(word_freq)

{'airport': 5, 'task': 10, 'force': 10, 'consider': 1, 'possible': 3, 'actions': 1, 'abate': 1, 'noise': 10, 'issues': 5, 'members': 3, 'southern': 10, 'san': 8, 'fernando': 7, 'valley': 12, 'airplane': 2, 'little': 1, 'month': 2, 'develop': 1, 'recommendations': 9, 'address': 2, 'aircraft': 2, 'affecting': 1, 'south': 5, 'five': 1, 'meetings': 3, 'hearing': 1, 'various': 2, 'neighborhood': 3, 'groups': 7, 'experts': 1, 'aviation': 5, 'consultant': 2, 'hmmh': 9, 'inc': 2, 'facilitator': 1, 'spent': 1, 'majority': 1, 'wednesday': 1, 'night': 1, 'distilling': 1, 'brought': 1, 'forward': 1, 'far': 1, 'feasibility': 1, 'implementing': 1, 'worked': 2, 'kevin': 1, 'karpe': 2, 'expert': 1, 'air': 10, 'traffic': 10, 'controller': 1, 'hollywood': 9, 'burbank': 12, 'federal': 1, 'administration': 1, 'california': 6, 'terminal': 1, 'radar': 1, 'approach': 1, 'control': 2, 'analyze': 1, 'departing': 7, 'flight': 7, 'procedures': 5, 'airfield': 1, 'well': 2, 'van': 4, 'nuys': 4, 'center': 1, 'since

**Exercise 4: Sentiment analysis with spaCy.**

In [210]:
amz = pd.read_table('amazon_cells_labelled.txt')
imdb = pd.read_table('imdb_labelled.txt')
yelp = pd.read_table('yelp_labelled.txt')

In [211]:
combined_col=[amz, imdb, yelp]

In [212]:
for column_name in combined_col:
  column_name.columns=['Review','Label']

In [213]:
for column_name in combined_col:
  print(column_name.columns)

Index(['Review', 'Label'], dtype='object')
Index(['Review', 'Label'], dtype='object')
Index(['Review', 'Label'], dtype='object')


In [214]:
keys_df=['amazon','imdb','yelp']

In [215]:
comb_data = pd.concat(combined_col,keys=keys_df).reset_index()
comb_data

,level_0,level_1,Review,Label
0,amazon,0,"Good case, Excellent value.",1
1,amazon,1,Great for the jawbone.,1
2,amazon,2,Tied to charger for conversations lasting more...,0
3,amazon,3,The mic is great.,1
4,amazon,4,I have to jiggle the plug to get it to line up...,0
...,...,...,...,...
2740,yelp,994,I think food should have flavor and texture an...,0
2741,yelp,995,Appetite instantly gone.,0
2742,yelp,996,Overall I was not impressed and would not go b...,0
2743,yelp,997,"The whole experience was underwhelming, and I ...",0


In [216]:
comb_data=comb_data.drop('level_1',axis=1)
comb_data

,level_0,Review,Label
0,amazon,"Good case, Excellent value.",1
1,amazon,Great for the jawbone.,1
2,amazon,Tied to charger for conversations lasting more...,0
3,amazon,The mic is great.,1
4,amazon,I have to jiggle the plug to get it to line up...,0
...,...,...,...
2740,yelp,I think food should have flavor and texture an...,0
2741,yelp,Appetite instantly gone.,0
2742,yelp,Overall I was not impressed and would not go b...,0
2743,yelp,"The whole experience was underwhelming, and I ...",0


In [228]:
comb_data.isnull().sum()

level_0    0
Review     0
Label      0
dtype: int64

In [229]:
comb_data.to_csv("SAD.csv", index=False)

In [231]:
df = pd.read_csv('SAD.csv')
df

,level_0,Review,Label
0,amazon,"Good case, Excellent value.",1
1,amazon,Great for the jawbone.,1
2,amazon,Tied to charger for conversations lasting more...,0
3,amazon,The mic is great.,1
4,amazon,I have to jiggle the plug to get it to line up...,0
...,...,...,...
2740,yelp,I think food should have flavor and texture an...,0
2741,yelp,Appetite instantly gone.,0
2742,yelp,Overall I was not impressed and would not go b...,0
2743,yelp,"The whole experience was underwhelming, and I ...",0


In [232]:
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [234]:
nlp = spacy.load("en_core_web_sm")
stpwrds = list(STOP_WORDS)
stpwrds

['noone',
 'not',
 'was',
 'four',
 'everyone',
 '‘ve',
 'than',
 'together',
 'me',
 "'re",
 'should',
 'whereas',
 'between',
 'of',
 'give',
 'whether',
 'there',
 'nevertheless',
 'which',
 'here',
 'very',
 'take',
 'twenty',
 'within',
 'n’t',
 "'s",
 'thereafter',
 'five',
 'whereby',
 'perhaps',
 'besides',
 'has',
 'ca',
 'as',
 'all',
 'since',
 'yet',
 'nowhere',
 'whence',
 'be',
 'them',
 'made',
 'would',
 'first',
 'for',
 'amongst',
 'else',
 'sometime',
 'part',
 'afterwards',
 'same',
 'quite',
 'therefore',
 'most',
 're',
 'have',
 'might',
 'wherein',
 'elsewhere',
 'nor',
 'nine',
 'though',
 'nobody',
 'you',
 'after',
 'rather',
 'whereupon',
 'front',
 'because',
 'my',
 'becoming',
 'seems',
 'under',
 'seemed',
 'while',
 "'m",
 'thus',
 'thereupon',
 'how',
 'bottom',
 'only',
 'she',
 'hundred',
 'somewhere',
 'via',
 'an',
 'formerly',
 'using',
 'a',
 'it',
 'always',
 'another',
 'mostly',
 'put',
 'did',
 'may',
 'her',
 'eleven',
 'seem',
 'further',
 

In [235]:
X = df['Review'] 
ylabels = df['Label'] 


X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2,random_state = 0)

In [236]:
# transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

nltk.download('stopwords')
stopwords=nltk.corpus.stopwords.words('english')
def clean_text(text):
    edited_text=re.sub('\W'," ",text) 
    edited_text=re.sub("  "," ",edited_text)
    edited_text=edited_text.split(" ")
    edited_text=" ".join([char for char in edited_text if char!= ""])
    edited_text=edited_text.lower()
    edited_text=re.sub('\d+',"",edited_text)
    edited_text=re.split('\W+',edited_text)
    edited_text=" ".join([word for word in edited_text if word not in stopwords])
    return edited_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [238]:
pat = r'[\d.,]+|[A-Z][.A-Z]+\b\.*|\w+|\S'
tokenizer = RegexpTokenizer(pat)
bow_v = TfidfVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = tokenizer.tokenize)
classifier = LinearSVC()

In [239]:
p = Pipeline([("cleaner", predictors()),('vectorizer', bow_v),('classifier', classifier)])
p.fit(X_train,y_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x7fd6e85de4d0>),
                ('vectorizer',
                 TfidfVectorizer(stop_words='english',
                                 tokenizer=<bound method RegexpTokenizer.tokenize of RegexpTokenizer(pattern='[\\d.,]+|[A-Z][.A-Z]+\\b\\.*|\\w+|\\S', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)>)),
                ('classifier', LinearSVC())])

In [240]:
predictions = p.predict(X_test)
predictions

array([0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,

In [241]:
accuracy_score=accuracy_score(y_test,predictions)
accuracy_score

0.7923497267759563